In [3]:
import torch
import numpy as np


In [3]:
data = torch.load('pattern_matrices.pt')
print(len(data), data[0].keys())

2000 dict_keys(['prompt_text', 'prompt_token_ids', 'prompt_attention_mask', 'token_ids', 'token_pattern_matrices'])


# 1. 统计数据集信息

1. prompt 统计信息

In [12]:
prompt_lens = np.array([len(sample['prompt_token_ids']) for sample in data.values()])
prompt_lens.min(), prompt_lens.max(), prompt_lens.mean(), prompt_lens.std()

(41, 382, 65.696, 30.360823177246033)

2. decoding 统计信息

In [22]:
decoding_tokens = []
for i in range(len(data)):
    sample = data[i]
    decoding_tokens.append(sample['token_ids'][prompt_lens[i]:])
decoding_tokens = torch.stack(decoding_tokens)
print(decoding_tokens.shape)
decoding_token_lens = np.array([len(x) for x in decoding_tokens])
decoding_token_lens.min(), decoding_token_lens.max(), decoding_token_lens.mean(), decoding_token_lens.std()

torch.Size([2000, 64])


(64, 64, 64.0, 0.0)

In [23]:
decoding_token_pattern_matrices = []
for i in range(len(data)):
    sample = data[i]
    decoding_token_pattern_matrices.append(sample['token_pattern_matrices'][prompt_lens[i]:]) # (#decoding_tokens, #layers, #experts)
decoding_token_pattern_matrices = torch.stack(decoding_token_pattern_matrices)
decoding_token_pattern_matrices.shape, decoding_token_pattern_matrices[0].shape, decoding_token_pattern_matrices[0][0]

(torch.Size([2000, 63, 32, 8]),
 torch.Size([63, 32, 8]),
 tensor([[1., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1., 1., 0.],
         [0., 0., 0., 0., 1., 1., 0., 0.],
         [1., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 1., 0., 0.],
         [0., 0., 0., 1., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0., 1., 0.],
         [0., 0., 1., 0., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 1., 0.],
         [1., 0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 1., 0.],
         [0., 0., 0., 0., 1., 0., 0., 1.],
         [0., 1., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0., 1., 0., 0

# 构建 Predictor 训练集

In [57]:
from datasets import Dataset
hf_data = {
    'prompt_text': [],
    'prompt_tokens_len': [],
    'token_ids': [],
    # 'prompt_token_ids': [],
    # 'decoding_token_ids': [],
    'token_pattern_matrices': []
}

# for i in range(10):
for i in range(len(data)):
    sample = data[i]
    prompt_text = sample['prompt_text']
    padded_prompt_token_ids = sample['prompt_token_ids']
    prompt_attention_mask = sample['prompt_attention_mask']
    start_index = prompt_attention_mask.argmax().item()
    token_ids = sample['token_ids'][start_index:-1]
    prompt_token_ids = padded_prompt_token_ids[start_index:]
    prompt_tokens_len = len(prompt_token_ids)
    decoding_token_ids = sample['token_ids'][len(prompt_attention_mask):-1]
    token_pattern_matrices = sample['token_pattern_matrices'][start_index:]
    assert len(token_ids)==len(decoding_token_ids)+len(prompt_token_ids)
    assert token_ids.numpy().tolist()==prompt_token_ids.numpy().tolist()+decoding_token_ids.numpy().tolist()
    assert len(token_pattern_matrices)==len(token_ids)
    hf_data['prompt_text'].append(prompt_text)
    hf_data['prompt_tokens_len'].append(prompt_tokens_len)
    hf_data['token_ids'].append(token_ids)
    # hf_data['prompt_token_ids'].append(prompt_token_ids)
    # hf_data['decoding_token_ids'].append(decoding_token_ids)
    hf_data['token_pattern_matrices'].append(token_pattern_matrices)
hf_data = Dataset.from_dict(hf_data)
hf_data

Dataset({
    features: ['prompt_text', 'prompt_tokens_len', 'token_ids', 'token_pattern_matrices'],
    num_rows: 2000
})

In [58]:
prompt_token_ids.shape, prompt_attention_mask, prompt_attention_mask.sum(), prompt_attention_mask.argmax().item(), decoding_token_ids.shape, token_ids.shape, token_pattern_matrices.shape

(torch.Size([382]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        

In [59]:
hf_data.push_to_hub('marsggbo/mixtral_8x7b_moe_alpaca_2k_token_pattern')

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.94s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/marsggbo/mixtral_8x7b_moe_alpaca_2k_token_pattern/commit/2fc2ba69bd0cf6b5f5dba433d10f5641ba53048f', commit_message='Upload dataset', commit_description='', oid='2fc2ba69bd0cf6b5f5dba433d10f5641ba53048f', pr_url=None, pr_revision=None, pr_num=None)

In [61]:
from datasets import load_dataset

dataset = load_dataset("marsggbo/mixtral_8x7b_moe_alpaca_2k_token_pattern")
dataset

Generating train split: 100%|██████████| 2000/2000 [00:01<00:00, 1911.88 examples/s]


DatasetDict({
    train: Dataset({
        features: ['prompt_text', 'prompt_tokens_len', 'token_ids', 'token_pattern_matrices'],
        num_rows: 2000
    })
})

In [66]:
from datasets import Dataset

In [63]:
lens = [x for x in dataset['train']['prompt_tokens_len']]
print(np.min(lens), np.max(lens), np.mean(lens), np.std(lens))
# lens = [len(x) for x in dataset['test']['prompt_token_ids']]
# print(np.min(lens), np.max(lens), np.mean(lens), np.std(lens))

37 382 59.2955 21.39596176267849


In [69]:

from typing import List, Optional, Tuple, Union
Union[str, Dataset]

typing.Union[str, datasets.arrow_dataset.Dataset]

In [65]:
sample = dataset['train'][0]
prompt_tokens_len = sample['prompt_tokens_len']
len(sample['token_ids'][:prompt_tokens_len]), np.stack(sample['token_pattern_matrices']).shape

(39, (102, 32, 8))

In [77]:
labels = np.stack(sample['token_pattern_matrices'])
labels = torch.from_numpy(labels).int()
labels

tensor([[[0, 1, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 1],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 1]],

        [[0, 0, 0,  ..., 1, 1, 0],
         [1, 0, 1,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 1, 0]],

        [[0, 0, 0,  ..., 1, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 1, 0, 0],
         ...,
         [1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 1,  ..., 0, 1, 0]],

        ...,

        [[0, 0, 1,  ..., 1, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 1],
         ...,
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 1, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 1, 0]],

        [[1, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 1, 0, 1],
         [1,

In [73]:
torch.tensor(sample['token_ids'], dtype=int)

tensor([    1, 20811,   349,   396, 13126,   369, 13966,   264,  3638, 28723,
        12018,   264,  2899,   369,  6582,  1999,  2691,   274,   272,  2159,
        28723,    13,    13, 27332,  3133,  3112, 28747,    13, 28784,   648,
        28705, 28770,   327,  1550,    13,    13, 27332, 12107, 28747,    13,
        28784,   648, 28705, 28770, 21588, 28705, 28774, 28723,     2, 10020,
        28744,     2, 10020, 28744,    13,    13, 27332, 11530, 12107, 28747,
           13, 28774, 28723,     2, 10020, 28744,     2, 10020, 28744,    13,
           13, 27332,  1529, 11009,   352, 28747,    13,  7477,   368,   967,
        28705, 28784,   304, 28705, 28770,  2553, 28725,   368,   625, 28705,
        28774, 28723,   415,  1474, 28705, 28774,   349,   272,  2648,   302,
        28705, 28784])

In [15]:
def acc_precision_recall_f1(y_true_origin, y_pred_origin):
    bs, seq_len, num_layer, num_experts = y_true_origin.shape
    y_true = np.reshape(y_true_origin, (bs, seq_len, num_layer * num_experts))
    y_pred = np.reshape(y_pred_origin, (bs, seq_len, num_layer * num_experts))
    y_true = np.transpose(y_true, (1, 0, 2)) # (seq, bs, num_layer * num_experts)
    y_pred = np.transpose(y_pred, (1, 0, 2)) # (seq, bs, num_layer * num_experts)
    y_true = (y_true.sum(1)>0).astype(int) # (seq, num_layer * num_experts)
    y_pred = (y_pred.sum(1)>0).astype(int) # (seq, num_layer * num_experts)
    print(y_true.shape, y_true)
    # 真正例 (True Positives)
    TP = np.sum((y_true == 1) & (y_pred == 1))
    
    # 假正例 (False Positives)
    FP = np.sum((y_true == 0) & (y_pred == 1))
    
    # 假负例 (False Negatives)
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    # 真负例 (True Negatives)
    TN = np.sum((y_true == 0) & (y_pred == 0))

    y_true = y_true.reshape(-1, 256)
    y_pred = y_pred.reshape(-1, 256)
    print(f"origin y_true.shape={y_true.shape}")
    indices = np.any(y_true, axis=-1)
    print(indices.shape)
    y_true = y_true[indices]
    y_pred = y_pred[indices]
    print(f"filtered y_true.shape={y_true.shape}")

    # 准确率
    num_tokens = y_true.shape[0]
    accuracy = TP / (num_tokens*64)
    recall = 0
    precision = 0
    f1 = 0
    print(f"non-padding ratio: {indices.sum()}/{len(indices)}={indices.sum()/len(indices)}\n")

    return {
        'accuracy': accuracy,
        'recall': recall,
        'precision': precision,
        'f1': f1,
    }

In [16]:
seq_len = 12
pad_seq_len = 4 
y_true = torch.randint(0,2,(4,8,32,8)).numpy()
y_pred = torch.randint(0,2,(4,8,32,8)).numpy()
print(acc_precision_recall_f1(y_true, y_pred))

(8, 256) [[1 1 1 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 0]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
origin y_true.shape=(8, 256)
(8,)
filtered y_true.shape=(8, 256)
non-padding ratio: 8/8=1.0

{'accuracy': 3.46875, 'recall': 0, 'precision': 0, 'f1': 0}


In [6]:
dataset_names = {
    "auto_categorization": 328,
    "tense": 286,
    "disfl_qa": 8000,
    "semantic_parsing_in_context_sparc": 1160,
    "word_sorting": 1900,
    "linguistics_puzzles": 2000,
}

In [14]:
import datasets
dataset_name = "tasksource/bigbench"
# names = datasets.get_dataset_config_names(dataset_name)

names = list(dataset_names.keys())
all_inputs = []
for name in names:
    print(name)
    all_inputs.append(datasets.load_dataset(dataset_name, name))

auto_categorization


/home/nus-hx/.conda/envs/moe/lib/python3.9/site-packages/datasets/load.py:1461: FutureWarning: The repository for tasksource/bigbench contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tasksource/bigbench
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


tense
disfl_qa
semantic_parsing_in_context_sparc
word_sorting
linguistics_puzzles


In [15]:
all_inputs[0]

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'multiple_choice_targets', 'multiple_choice_scores', 'idx'],
        num_rows: 263
    })
    validation: Dataset({
        features: ['inputs', 'targets', 'multiple_choice_targets', 'multiple_choice_scores', 'idx'],
        num_rows: 65
    })
})

In [18]:

train_all_inputs = []
valid_all_inputs = []
for dataset in all_inputs:
    train_all_inputs += [text for text in dataset["train"]["inputs"]]
    valid_all_inputs += [text for text in dataset["validation"]["inputs"]]
len(train_all_inputs), len(valid_all_inputs)

(10936, 2733)

In [1]:
import torch
import numpy as np
import datasets

/home/nus-hx/.conda/envs/moe/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = datasets.load_dataset("marsggbo/bigbench4switch64_pattern_predictor")['train']
dataset

Dataset({
    features: ['prompt_text', 'prompt_ids', 'decode_ids', 'prompt_pattern', 'decode_pattern'],
    num_rows: 10936
})

In [3]:
np.array(dataset[0]['decode_pattern']).shape

(6, 32)

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base")
tokenizer.padding_side = 'left'
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base")
model.lm_head = torch.nn.Linear(768, 6*64)
model = model.cuda(1)

In [5]:
model.__class__, model.config.hidden_size

(transformers.models.t5.modeling_t5.T5ForConditionalGeneration, 768)

In [6]:
params = sum([p.numel() for p in model.parameters()])
params*2/1024**2

425.718017578125

In [13]:
import time
decoder_lens = list(range(1, 16))
bs = 32
encoder_len = 512
device = torch.device("cuda:1")
time_costs = {}
# warmup

input_ids = torch.randint(0, 100, (bs, encoder_len)).to(device)
attention_mask = torch.ones(bs, encoder_len).to(device)
decoder_input_ids =torch.randint(0, 100, (bs, 10)).to(device)
out = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    decoder_input_ids=decoder_input_ids,
)

for decoder_len in decoder_lens:
    input_ids = torch.randint(0, 100, (bs, encoder_len)).to(device)
    attention_mask = torch.ones(bs, encoder_len).to(device)
    decoder_input_ids =torch.randint(0, 100, (bs, decoder_len)).to(device)
    torch.cuda.synchronize()
    start = time.time()
    for i in range(10):
        out = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
        )
    torch.cuda.synchronize()
    end = time.time()
    time_costs[decoder_len] = (end-start)/10
time_costs

{1: 0.2365039348602295,
 2: 0.2400728702545166,
 3: 0.24014568328857422,
 4: 0.24187352657318115,
 5: 0.2428675889968872,
 6: 0.24388659000396729,
 7: 0.2439492702484131,
 8: 0.2441192626953125,
 9: 0.24286985397338867,
 10: 0.24423601627349853,
 11: 0.24433786869049073,
 12: 0.24466307163238527,
 13: 0.24510047435760499,
 14: 0.2453702926635742,
 15: 0.24580740928649902}

In [53]:
bs = 32
batch_indices = [list(range(i,i+bs)) for i in range(0, len(dataset), bs)]
all_prompt_text = np.array(dataset['prompt_text'])
all_decode_ids = np.array(dataset['decode_ids'])
all_decode_pattern = np.array(dataset['decode_pattern'])
num_experts_per_layer = 64
for indices in batch_indices[:4]:
    batch_text = all_prompt_text[indices].tolist()
    data = tokenizer(batch_text, return_tensors="pt", return_attention_mask=True, padding=True)
    input_ids = data.input_ids.cuda()
    attention_mask = data.attention_mask.cuda()
    decoder_input_ids = torch.tensor(all_decode_ids[indices]).cuda()
    decode_pattern = torch.tensor(all_decode_pattern[indices]).permute(0,2,1).cuda()
    decode_pattern = torch.nn.functional.one_hot(decode_pattern, num_classes=num_experts_per_layer).float()
    out = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
    )
    logits = out.logits
    loss = torch.nn.functional.binary_cross_entropy_with_logits(
            logits.view(-1, num_experts_per_layer),
            decode_pattern.view(-1, num_experts_per_layer),
            reduction='mean')
    print(out.logits.shape, loss)

torch.Size([8, 32, 384]) tensor(0.6928, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([8, 32, 384]) tensor(0.6928, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([8, 32, 384]) tensor(0.6927, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
torch.Size([8, 32, 384]) tensor(0.6928, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [41]:
out.logits.shape

torch.Size([8, 32, 32128])

In [36]:
decode_pattern.shape, decoding_token_ids.shape

(torch.Size([8, 32, 6]), torch.Size([8, 32]))

In [2]:
import torch
import torch.nn.functional as F

def prepare_batch(input_seqs, target_seqs, pad_token_id=0):
    # 输入和目标序列的填充
    input_padded = torch.nn.utils.rnn.pad_sequence(
        input_seqs[::-1], batch_first=True, padding_value=pad_token_id).flip(dims=[1])
    target_padded = torch.nn.utils.rnn.pad_sequence(
        target_seqs, batch_first=True, padding_value=pad_token_id)

    # 创建编码器和解码器的掩码
    input_mask = (input_padded != pad_token_id)
    target_mask = (target_padded != pad_token_id)

    return input_padded, input_mask, target_padded, target_mask

# 假设 input_seqs 和 target_seqs 是预处理后的序列列表，其中每个元素是一个tensor
input_seqs = [torch.tensor([1, 2, 3, 4, 5, 6]), torch.tensor([4, 5])]
target_seqs = [torch.tensor([1, 2]), torch.tensor([3, 4, 5, 6, 7, 8])]

input_padded, input_mask, target_padded, target_mask = prepare_batch(input_seqs, target_seqs)
(input_padded.shape, input_mask.shape, target_padded.shape, target_mask.shape)
(input_padded, input_mask, target_padded, target_mask)

(tensor([[0, 0, 0, 0, 5, 4],
         [6, 5, 4, 3, 2, 1]]),
 tensor([[False, False, False, False,  True,  True],
         [ True,  True,  True,  True,  True,  True]]),
 tensor([[1, 2, 0, 0, 0, 0],
         [3, 4, 5, 6, 7, 8]]),
 tensor([[ True,  True, False, False, False, False],
         [ True,  True,  True,  True,  True,  True]]))

In [36]:
bs=2
seq_len = torch.randint(1, 10, (bs,))
target_seqs = [torch.randint(0,100,(seq_len[i], )) for i in range(bs)]
pad_target_seqs = torch.nn.utils.rnn.pad_sequence(target_seqs, batch_first=True, padding_value=0)
print([x.shape for x in target_seqs])
print(pad_target_seqs.shape, pad_target_seqs)


labels = [torch.randint(0, 2, (seq_len[i], 16)) for i in range(bs)]
print([x.shape for x in labels])
pad_labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)
print(pad_labels.shape)

[torch.Size([9]), torch.Size([3])]
torch.Size([2, 9]) tensor([[52, 96, 67, 19, 30, 99, 36, 52, 34],
        [55, 53, 64,  0,  0,  0,  0,  0,  0]])
[torch.Size([9, 16]), torch.Size([3, 16])]
torch.Size([2, 9, 16])


In [37]:
pad_labels[1]

tensor([[0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1],
        [0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0],
        [0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])